In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# %cd /gdrive/My Drive/Colab_ML/PokeGan
# !git clone https://github.com/moxiegushi/pokeGAN

In [3]:
import os
import tensorflow as tf
import numpy as np
import cv2
import random 
import scipy
%cd "/content/drive/My Drive/Colab_ML/PokeGan"
%pwd
from utils import *
import datetime

/content/drive/My Drive/Colab_ML/PokeGan


In [0]:
slim = tf.contrib.slim

HEIGHT, WIDTH, CHANNEL = 128,128,3
BATCH_SIZE = 64
EPOCH = 5000

version = 'newPokemon'
newPoke_path = '/content/drive/My Drive/Colab_ML/PokeGan/'+version

In [0]:
def lrelu(x,n,leak = 0.2):
    return tf.maximum(x,leak*x,name = n)

In [0]:
def process_data():
    current_dir = os.getcwd()
    pokemon_dir = os.path.join(current_dir, 'data')
    images = []
    for each in os.listdir(pokemon_dir):
        images.append(os.path.join(pokemon_dir, each))
    
    all_images = tf.convert_to_tensor(images, dtype = tf.string)

    images_queue = tf.train.slice_input_producer([all_images])
    content = tf.read_file(images_queue[0])
    image = tf.image.decode_jpeg(content,channels = CHANNEL)

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image , max_delta = 0.1)
    image = tf.image.random_contrast(image, lower = 0.9, upper = 1.1)

    size = [HEIGHT, WIDTH]
    image = tf.image.resize_images(image, size)
    image.set_shape([HEIGHT, WIDTH, CHANNEL])
    image = tf.cast(image, tf.float32)

    image = image/255.0

    images_batch = tf.train.shuffle_batch([image],batch_size=BATCH_SIZE, num_threads = 4, capacity = 200+3*BATCH_SIZE, min_after_dequeue = 200)
    num_images = len(images)

    return images_batch, num_images

In [0]:
def generator(input, random_dim, is_train, reuse=False):
    print('Gene')
    c4, c8, c16, c32, c64 = 512, 256, 128, 64, 32 # channel num
    s4 = 4
    output_dim = CHANNEL  # RGB image
    with tf.variable_scope('gen') as scope:
        if reuse:
            scope.reuse_variables()
        w1 = tf.get_variable('w1', shape=[random_dim, s4 * s4 * c4], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[c4 * s4 * s4], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))
        flat_conv1 = tf.add(tf.matmul(input, w1), b1, name='flat_conv1')
        # 4*4*512
        conv1 = tf.reshape(flat_conv1, shape=[-1, s4, s4, c4], name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
        # 8*8*256
        conv2 = tf.layers.conv2d_transpose(act1, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
        # 16*16*128
        conv3 = tf.layers.conv2d_transpose(act2, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
        # 32*32*64
        
        
        conv4 = tf.layers.conv2d_transpose(act3, c32, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
        # 64*64*32
        
        conv5 = tf.layers.conv2d_transpose(act4, c64, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
        
        #128*128*3
        conv6 = tf.layers.conv2d_transpose(act5, output_dim, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                           name='conv6')
        # bn6 = tf.contrib.layers.batch_norm(conv6, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn6')
        act6 = tf.nn.tanh(conv6, name='act6')
        return act6

In [0]:
def discriminator(input, is_train, reuse=False):
    print('Disc')
    c2, c4, c8, c16 = 64, 128, 256, 51
    with tf.variable_scope('dis') as scope:
        if reuse:
            scope.reuse_variables()
        conv1 = tf.layers.conv2d(input, c2, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv1')
        act1 = lrelu(conv1, n='act1')
        conv2 = tf.layers.conv2d(act1, c4, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn2')
        act2 = lrelu(bn2, n='act2')
        
        
        conv3 = tf.layers.conv2d(act2, c8, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn3')
        act3 = lrelu(bn3, n='act3')
        
        
        conv4 = tf.layers.conv2d(act3, c16, kernel_size=[5, 5], strides=[2, 2], padding="SAME",
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay = 0.9,  updates_collections=None, scope='bn4')
        act4 = lrelu(bn4, n='act4')
                
                
        
        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')
        
        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                             initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32,
                             initializer=tf.constant_initializer(0.0))

        
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
        
        acted_out = tf.nn.sigmoid(logits)
        return logits 




In [0]:
def train():

    random_dim = 100

    
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        is_train = tf.placeholder(tf.bool, name='is_train')
    

    fake_image = generator(random_input, random_dim, is_train)
    real_result = discriminator(real_image, is_train)
    fake_result = discriminator(fake_image, is_train, reuse=True)
    
    d_loss = tf.reduce_mean(fake_result) - tf.reduce_mean(real_result)
    g_loss = -tf.reduce_mean(fake_result) 


    t_vars = tf.trainable_variables()
    d_vars = [var for var in t_vars if 'dis' in var.name]
    g_vars = [var for var in t_vars if 'gen' in var.name]


    trainer_d = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(d_loss, var_list=d_vars)
    trainer_g = tf.train.RMSPropOptimizer(learning_rate=2e-4).minimize(g_loss, var_list=g_vars)

    d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in d_vars]

    
    batch_size = BATCH_SIZE
    image_batch, samples_num = process_data()
    
    batch_num = int(samples_num / batch_size)
    total_batch = 0
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    ckpt = tf.train.latest_checkpoint('/content/drive/My Drive/Colab_ML/PokeGan/model/' + version)
    try:
        print('[INFO] RESTORED')
        saver.restore(sess, ckpt)
        


    except:
        pass
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    print ('total training sample num:%d' % samples_num)
    print ('batch size: %d, batch num per epoch: %d, epoch num: %d' % (batch_size, batch_num, EPOCH))
    print ('start training...')
    for i in range(1950,EPOCH):
        print('epochh')
        for j in range(batch_num):
            print('batch')
            d_iters = 5
            g_iters = 1

            train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            for k in range(d_iters):
                train_image = sess.run(image_batch)
                sess.run(d_clip)
                
                _, dLoss = sess.run([trainer_d, d_loss],
                                    feed_dict={random_input: train_noise, real_image: train_image, is_train: True})

            for k in range(g_iters):
                _, gLoss = sess.run([trainer_g, g_loss],
                                    feed_dict={random_input: train_noise, is_train: True})

            

        if i%50 == 0:
            with open('/content/drive/My Drive/Colab_ML/PokeGan/saverfile.txt','a+') as f:
                f.write('Saving image and model at {}\n'.format(str(datetime.datetime.now())))
            if not os.path.exists('/content/drive/My Drive/Colab_ML/PokeGan/model/' + version):
                os.makedirs('/content/drive/My Drive/Colab_ML/PokeGan/model/' + version)
            saver.save(sess, '/content/drive/My Drive/Colab_ML/PokeGan/model/' +version + '/' + str(i)) 
            if not os.path.exists(newPoke_path):
                os.makedirs(newPoke_path)
            sample_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
            imgtest = sess.run(fake_image, feed_dict={random_input: sample_noise, is_train: False})
            save_images(imgtest, [8,8] ,newPoke_path + '/epoch' + str(i) + '.jpg')
            
        print ('train:[%d],d_loss:%f,g_loss:%f' % (i, dLoss, gLoss))
    coord.request_stop()
    coord.join(threads)



In [10]:
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [11]:
if __name__=="__main__":

    train()

Gene
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
Disc
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Disc
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(in

train:[1950],d_loss:-35.540665,g_loss:18.063749
epochh
batch
batch
batch
batch
batch
train:[1951],d_loss:-25.432119,g_loss:12.724567
epochh
batch
batch
batch
batch
batch
train:[1952],d_loss:-32.875450,g_loss:14.857151
epochh
batch
batch
batch
batch
batch
train:[1953],d_loss:-34.828072,g_loss:17.929506
epochh
batch
batch
batch
batch
batch
train:[1954],d_loss:-32.977459,g_loss:18.150337
epochh
batch
batch
batch
batch
batch
train:[1955],d_loss:-29.344603,g_loss:14.836653
epochh
batch
batch
batch
batch
batch
train:[1956],d_loss:-33.911221,g_loss:17.765453
epochh
batch
batch
batch
batch
batch
train:[1957],d_loss:-20.486084,g_loss:5.785151
epochh
batch
batch
batch
batch
batch
train:[1958],d_loss:-33.549690,g_loss:17.576693
epochh
batch
batch
batch
batch
batch
train:[1959],d_loss:-32.099670,g_loss:15.916659
epochh
batch
batch
batch
batch
batch
train:[1960],d_loss:-34.446407,g_loss:18.223806
epochh
batch
batch
batch
batch
batch
train:[1961],d_loss:-35.774475,g_loss:18.069057
epochh
batch
batch

train:[2000],d_loss:-23.439814,g_loss:14.598978
epochh
batch
batch
batch
batch
batch
train:[2001],d_loss:-33.678207,g_loss:17.873051
epochh
batch
batch
batch
batch
batch
train:[2002],d_loss:-34.953484,g_loss:18.131298
epochh
batch
batch
batch
batch
batch
train:[2003],d_loss:-15.178753,g_loss:13.172270
epochh
batch
batch
batch
batch
batch
train:[2004],d_loss:-35.029541,g_loss:17.844116
epochh
batch
batch
batch
batch
batch
train:[2005],d_loss:-34.842716,g_loss:17.445408
epochh
batch
batch
batch
batch
batch
train:[2006],d_loss:-34.787582,g_loss:17.997244
epochh
batch
batch
batch
batch
batch
train:[2007],d_loss:-32.986702,g_loss:17.761524
epochh
batch
batch
batch
batch
batch
train:[2008],d_loss:-30.053185,g_loss:18.050495
epochh
batch
batch
batch
batch
batch
train:[2009],d_loss:-32.542393,g_loss:18.278393
epochh
batch
batch
batch
batch
batch
train:[2010],d_loss:-33.890018,g_loss:15.673803
epochh
batch
batch
batch
batch
batch
train:[2011],d_loss:-34.754997,g_loss:17.991116
epochh
batch
batc

KeyboardInterrupt: ignored